In [1]:
import src.SessionProcessor as SessionProcessor
from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache

def compare_elements(arr1, arr2):
        arr1 = arr1.flat
        arr2 = arr2.flat
        num_elements = len(arr1)
        
        # If there are the same number of elements in both arrays, check that the arrays have identical elements
        if num_elements == len(arr2):
            for idx in range(num_elements):
                if arr1[idx] != arr2[idx]:
                    return False
        else:
            return False
        
        return True

def assertFalse(arg):
    if arg:
        raise AssertionError()

def assertTrue(arg):
    if not(arg):
        raise AssertionError()


C:\Users\Demogorgon\anaconda3\envs\Python37\lib\site-packages\allensdk\brain_observatory\comparison_utils.py:9: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
# Everything here is required to correctly load a session object. Session objects take a while
# to load (a touch over a minute on my machine), so only one session is loaded to make the tests
# go faster.
# It'll take a couple minutes to run these tests, even though there aren't many, because of the
# session object.
data_root = "C:/Users/Demogorgon/Documents/College/Marcus/Boston University PhD/Ocker Lab"
manifest_path = f"{data_root}/AllenSDK_Data/manifest.json"
stim = 'drifting_gratings_contrast'
session_id = 778240327
cache = EcephysProjectCache(manifest=manifest_path)
session = cache.get_session_data(session_id=session_id)

# We're comparing shuffled and unshuffled PSTH data, so we need a shuffled and unshuffled version
# of the session
processor = SessionProcessor.SessionProcessor(session)
unshuffled = processor.construct_decoder('drifting_gratings_contrast', 'orientation')
shuffled = processor.construct_decoder('drifting_gratings_contrast', 'orientation', shuffle_trials=True)

In [3]:
# The PSTHs I'm refering to here are each cell's spike count after a stim presentation.
# For both shuffled and unshuffled trials, those are stored in:
# processor._decoders[name].x
unshuffled_x = processor._decoders[unshuffled].x
unshuffled_y = processor._decoders[unshuffled].y
shuffled_x = processor._decoders[shuffled].x
print(f"unshuffled_x.shape = {unshuffled_x.shape}")
print(f"unshuffled_y.shape = {unshuffled_y.shape}")
print(f"shuffled_x.shape = {shuffled_x.shape}")
print(f"axis 0 shape: {unshuffled_x.mean(axis=0).shape}")
print(f"axis 1 shape: {unshuffled_x.mean(axis=1).shape}")
print(f"axis 2 shape: {unshuffled_x.mean(axis=2).shape}")

try:
    assertFalse(compare_elements(unshuffled_x, shuffled_x))
    print("The data is being shuffled")
except AssertionError as e:
    print("The data is not being shuffled")

try:
    # PSTHs should be averaged over presentation, which should be axis=0
    # num_presentations, num_bins, num_units = x.shape
    assertTrue(compare_elements(unshuffled_x.mean(axis=0), shuffled_x.mean(axis=0)))
    print("The trial averaged PSTHs of the shuffled and unshuffled data are identical")
except AssertionError as e:
    print("The trial averaged PSTHs of the shuffled and unshuffled data are NOT identical")

unshuffled_x.shape = (540, 11, 784)
unshuffled_y.shape = (540,)
shuffled_x.shape = (540, 11, 784)
axis 0 shape: (11, 784)
axis 1 shape: (540, 784)
axis 2 shape: (540, 11)
The data is being shuffled
The trial averaged PSTHs of the shuffled and unshuffled data are identical
